# Modelling I – fitting linear regression on Titanic data

This notebook contains an example of fitting and evaluating linear regression model on Titanic data. We will use tickets as modelling units (rows, entities), *fare* as target (possibly log fare) and various features as predictors.

## Data

We use the dataset Titanic and data preparation from the recent practice (see Data Preparation).

In [ ]:
# setup
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
import statsmodels.api as sm
import statsmodels.formula.api as smf

pd.set_option("display.precision", 2)

In [ ]:
# Titanic data reading and preparing - reminder from `Data Preparation` practice
df_t1 = pd.read_csv('titanic_train.csv') # adjust file path
df_t1 = df_t1[['passenger_id', 'ticket', 'pclass', 'fare', 'sex', 'age', 'cabin', 'embarked']]

# cleaning
df_t1 = df_t1[df_t1['fare'].notna() & (df_t1['fare']>0) & (df_t1['embarked'].notna())]

# making new dataset of tickets
# User function
def rate_males(s):
    return np.mean(np.where(s=='male', 1, 0))

### Base table
df_t2_base = df_t1[['ticket', 'pclass', 'fare']].drop_duplicates()
df_t2_base = df_t2_base.set_index('ticket') # setting 'ticket' column as key

### Multiple embarkment solution
df_t2_emb = df_t1.groupby('ticket').agg({'embarked': 'max'})
# no need to set index - groupby + agg sets index by default

### Some chosen features
df_t2_feat = df_t1.groupby('ticket').agg({'ticket': 'count', 'sex': [rate_males],
                                      'age': ['min', 'max', np.mean, 'count'], 'cabin': 'nunique'})
# column names update
df_t2_feat.columns = ['pass_cnt', 'rate_males', 'age_min', 'age_max', 'age_mean', 'age_valid_cnt', 'cabin_cnt']

# sex of the oldest person for the ticket
df_t2_feat_sex_oldest = df_t1.sort_values(by=['ticket', 'age'], ascending=[True, False]) \
    .drop_duplicates('ticket')[['ticket', 'sex']]
df_t2_feat_sex_oldest = df_t2_feat_sex_oldest.set_index('ticket') # setting 'ticket' column as key
df_t2_feat_sex_oldest.columns = ['sex_oldest']

### Joining tables together
df_t2 = df_t2_base.join(df_t2_emb) # join is by default LEFT and index<->index
df_t2 = df_t2.join(df_t2_feat)
df_t2 = df_t2.join(df_t2_feat_sex_oldest)

# mathematical transformations
df_t2['fare_log'] = np.log10(df_t2['fare']) # we use log10 for better interpretation, but simple log is ok, too.
df_t2['fare_per_pass'] = df_t2['fare'] / df_t2['pass_cnt']

# binning, making categories and flags
### pass_cnt
df_t2['pass_cnt_cat'] = pd.cut(df_t2['pass_cnt'], [0, 1, 2, 3, 1000], labels=['1', '2', '3', '4+'])

### age_mean
df_t2['age_mean_cat'] = pd.cut(df_t2['age_mean'], [0, 15, 20, 25, 30, 40, 1000],
                             labels=['15-', '15-20', '20-25', '25-30', '30-40', '40+'])

### cabin_cnt (same approach as pass_cnt)
df_t2['cabin_cnt_cat'] = pd.cut(df_t2['cabin_cnt'], [0, 1, 2, 1000], right=False, labels=['none', '1', '2+'])

# flags
df_t2['flag_child'] = (df_t2['age_min'] < 15)
df_t2['flag_baby'] = (df_t2['age_min'] < 3)

### cleanup
del df_t2_base
del df_t2_emb
del df_t2_feat
del df_t2_feat_sex_oldest

In [ ]:
df_t2

## Linear regression

We learned that *fare* is very skew, we have transformed it by log10. So we take *fare_log* as target and *embarked*, *pclass* and *pass_cnt* as predictors.

In [ ]:
g = sns.displot(data=df_t2, x="fare_log", kind="kde") \
    .set_axis_labels("Log10(ticket fare)", "Density") \
    .set(title="Distribution of log ticket fare")

In [ ]:
X = df_t2[['pass_cnt']]
y = df_t2['fare_log']

# fit model
modelA = LinearRegression().fit(X, y)

# get coefficients
print('Intercept: ', modelA.intercept_)
print('Beta coefficients: ', modelA.coef_)

Try to interpret coefficients.

How is the model performing? Let's go for a cross-validation and R2 score.

In [ ]:
scores = cross_val_score(LinearRegression(), X, y, cv=4)
print('R2 by cval: ', scores)

Not bad. Let's look at the coefficient significance.

In [ ]:
modelC = smf.ols("fare_log ~ pass_cnt", data=df_t2).fit()
print(modelC.summary()) # detailed information of model and coefficients

Ok. Try yourself to improve the model by adding *pclass*, *embarked* and maybe some other predictors.  
**Warning!** *pclass* is encoded as integer but in fact this is categorical predictor.